In [39]:
from stackapi import StackAPI
import pandas as pd
import time
SITE = StackAPI('stackoverflow')

In [40]:
start_time = int(time.mktime(time.strptime("2016-06-01T00:00:00", "%Y-%m-%dT%H:%M:%S")))
end_time = int(time.mktime(time.strptime("2017-06-01T00:00:00", "%Y-%m-%dT%H:%M:%S")))
questions = SITE.fetch('questions', fromdate=start_time, todate=end_time,
                       min=20, sort='votes')

In [41]:
qdf = pd.DataFrame.from_dict(questions)
i = 2
while questions['has_more'] == True and i < 10:
    questions = SITE.fetch('questions', fromdate=start_time, todate=end_time,
                       min=20, sort='votes', page=i)
    tdf = pd.DataFrame.from_dict(questions)
    qdf = pd.concat([qdf, tdf])
    i += 1 
qdf = qdf.reset_index()
qdf = qdf.to_csv("questions.csv")

StackAPIError: 

In [33]:
import ast
answerlist = []
qdf = pd.read_csv("questions.csv")
for index, row in qdf.iterrows():
    rd = ast.literal_eval(row['items'])
    if 'accepted_answer_id' in rd and 'user_id' in rd['owner']:
        answerlist.append(rd['accepted_answer_id'])

In [34]:
numiters = int(len(answerlist) / 99)
adf = {}
if numiters == 0:
    answers = SITE.fetch('/answers/{ids}', ids=answerlist)
    adf = pd.DataFrame.from_dict(answers)
else:
    answers = SITE.fetch('/answers/{ids}', ids=answerlist[:99])
    adf = pd.DataFrame.from_dict(answers)
    j = 99
    k = j + 99
    for i in range(0, numiters):
        answers = SITE.fetch('/answers/{ids}', ids=answerlist[j:k])
        temp = pd.DataFrame.from_dict(answers)
        adf = pd.concat([adf, temp])
        j = j + 99
        k = k + 99
adf = adf.reset_index()
adf = adf.to_csv("answers.csv")

In [35]:
#Only run above when needed.
import ast
import pandas as pd
userasklist = []
useranswerlist = []
questionlist = []
adf = pd.read_csv("answers.csv")
qdf = pd.read_csv("questions.csv")
dq = {}
dtag = {}
for index, row in qdf.iterrows():
    q = ast.literal_eval(row['items'])
    if 'accepted_answer_id' in q and 'user_id' in q['owner']:
        dq[q['question_id']] = q['owner']['user_id']
        if 'tags' in q:
            dtag[q['question_id']] = q['tags']
        else:
            dtag[q['question_id']] = []

In [89]:
da = {}
for index, row in adf.iterrows():
    q = ast.literal_eval(row['items'])
    if 'user_id' in q['owner']:
        da[q['question_id']] = q['owner']['user_id']
    else:
        #find a way to delete this
        da[q['question_id']] = "Null"

d = {"asked": pd.Series(dq.values(), index=dq.keys()), 
     "answered": pd.Series(da.values(), index=dq.keys()),
     "tags":pd.Series(dtag.values(), index=dq.keys())}
data = pd.DataFrame(d)
data = data.reset_index()
data = data.rename(columns = {'index':'question'})
print data

     question answered    asked  \
0    38516736  2257100   722393   
1    32204807  3465182  2745567   
2    33869832   743382  3415258   
3    34562061  1906307   771848   
4    33155087   481165  1298028   
5    32665616  1263565  5352996   
6    36438034  1048572   596041   
7    31553560   231977   862919   
8    34039834  5760605  3668891   
9    32780315  3627726  1415560   
10   32475166  3191557  5188921   
11   37945376    95060    95060   
12   37548066   819742    88522   
13   32714787  4615090  4615090   
14   36872230   335858  5619103   
15   33984558  1072809  3125476   
16   34930735  1430627   980929   
17   31424561  1751810  1751810   
18   31764532  5147646   306941   
19   30739806  1824898  3063619   
20   31782580  4998761  4693318   
21   38453335   984780  5333340   
22   38713405  6052290  2629219   
23   37812286  2227743  5393528   
24   37800001   743382  6461611   
25   34637896    16633  1014251   
26   34352140  3597276   365488   
27   32860237   1526

In [119]:
import networkx as nx
#work on this....
g = nx.from_pandas_dataframe(data, "asked", "answered", edge_attr="tags")
dataasked = data.set_index("asked")
for n, sd in g.nodes_iter(True):
    if n in dataasked.ix[:,1].tolist():
        print dataasked.loc[n,"tags"]
        sd['tags'] = dataasked.loc[n,"tags"]

[u'java', u'generics', u'java-8', u'type-inference']
asked
3999748    [html, css, css3, svg, css-shapes]
3999748    [html, css, css3, svg, css-shapes]
Name: tags, dtype: object


KeyError: 'the label [2541573] is not in the [index]'

In [60]:
#calcualte diameter.  We must pick a subset that is entirely connected and check that diameter.  
#We will use the biggest network
bn = sorted(nx.connected_components(g), key=len, reverse=True)[0]
g = g.subgraph(bn)
print nx.diameter(g)
print sorted(nx.degree_centrality(g).iteritems(), key=lambda(k, v): (-v, k))[0:9]

4
[(217408, 0.7333333333333333), (2069064, 0.26666666666666666), (4332023, 0.13333333333333333), (550975, 0.06666666666666667), (747412, 0.06666666666666667), (1193781, 0.06666666666666667), (1333012, 0.06666666666666667), (1798031, 0.06666666666666667), (2927965, 0.06666666666666667)]


In [61]:
#change graph to just the largest network
so_nodes = g.nodes()
so_edges = g.edges()

In [45]:
from graphlab import SGraph, Vertex, Edge, canvas
canvas.set_target('ipynb')
so_graph = SGraph()
nodelist = []
for i in so_nodes:
    nodelist.append(Vertex(i))
so_graph = so_graph.add_vertices(nodelist)
for i, j in so_edges:
    so_graph = so_graph.add_edges(Edge(i,j))
so_graph.show(vlabel='id', arrows=True)

In [62]:
print nx.get_node_attributes(g , 'tags')

{}
